# Illustrating common terms usage using Wikinews in english

## getting data

We get the cirrussearch dump of wikinews (a dump meant for elastic-search indexation).

In [1]:
LANG="english"

In [2]:
%%bash

fdate=20170327
fname=enwikinews-$fdate-cirrussearch-content.json.gz
if [ ! -e  $fname ]
then
    wget "https://dumps.wikimedia.org/other/cirrussearch/$fdate/$fname"
fi


In [3]:
# iterator
import gzip
import json

FDATE = 20170327
FNAME = "enwikinews-%s-cirrussearch-content.json.gz" % FDATE

def iter_texts(fpath=FNAME):
    with gzip.open(fpath, "rt") as f:
        for l in f:
            data = json.loads(l)
            if "title" in data:
                yield data["title"]
                yield data["text"]

In [4]:
# also prepare nltk
import nltk
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Preparing data

we arrange the corpus as required by gensim

In [5]:
# make a custom tokenizer
import re
from nltk.tokenize import sent_tokenize
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer('\w[\w-]*|\d[\d,]*')

In [6]:
# prepare a text
def prepare(txt):
    # lower case
    txt = txt.lower()
    return [tokenizer.tokenize(sent) 
            for sent in sent_tokenize(txt, language=LANG)]

In [7]:
# we put all data in ram, it's not so much
corpus = []
for txt in iter_texts():
    corpus.extend(prepare(txt))

In [8]:
# how many sentences and words ?
words_count = sum(len(s) for s in corpus)
print("Corpus has %d words in %d sentences" % (words_count, len(corpus)))

Corpus has 1003521 words in 46159 sentences


## Testing bigram with and without common terms

The `Phrases` model gives us the possiblity of handling common terms, that is words that appears much time in a text and are there only to link objects between them.
While you could remove them, you may information, for *"the president is in america"* is not the same as *"the president of america"*

The common_terms parameter Phrases can help you deal with them in a smarter way, keeping them around but avoiding them to crush frequency statistics.

In [9]:
from gensim.models.phrases import Phrases

Using TensorFlow backend.


In [10]:
# which are the stop words we will use
from nltk.corpus import stopwords
" ".join(stopwords.words(LANG))

'i me my myself we our ours ourselves you your yours yourself yourselves he him his himself she her hers herself it its itself they them their theirs themselves what which who whom this that these those am is are was were be been being have has had having do does did doing a an the and but if or because as until while of at by for with about against between into through during before after above below to from up down in out on off over under again further then once here there when where why how all any both each few more most other some such no nor not only own same so than too very s t can will just don should now d ll m o re ve y ain aren couldn didn doesn hadn hasn haven isn ma mightn mustn needn shan shouldn wasn weren won wouldn'

In [11]:
# bigram std
%time bigram = Phrases(corpus)
# bigram with common terms
%time bigram_ct = Phrases(corpus, common_terms=stopwords.words(LANG))

CPU times: user 1.34 s, sys: 20 ms, total: 1.36 s
Wall time: 1.36 s
CPU times: user 1.6 s, sys: 24 ms, total: 1.62 s
Wall time: 1.62 s


### bigram with common terms inside

What are (some of) the bigram founds thanks to common terms

In [12]:
# grams that have more than 2 terms, are those with common terms
ct_ngrams = set((g[1], g[0].decode("utf-8"))
                     for g in bigram_ct.export_phrases(corpus) 
                     if len(g[0].split()) > 2)
ct_ngrams = sorted(list(ct_ngrams))
print(len(ct_ngrams), "grams with common terms found")
# highest scores
ct_ngrams[-20:]

564 grams with common terms found


[(7888.587500000001, 'christ of latter-day'),
 (8249.503267973856, 'skull and bones'),
 (8452.058035714284, 'preserved in amber'),
 (8624.480689245396, 'aisyah and doan'),
 (8664.810068649886, 'funded by your generous'),
 (9015.528571428571, 'restored as burkina'),
 (9964.531578947368, 'click on the donate'),
 (10178.822580645161, 'qatar of intervening'),
 (10380.724721819062, 'sinks in suva'),
 (11462.050213675215, 'lahm to hang'),
 (11485.911021233569, 'istanbul s ataturk'),
 (11686.796296296296, 'derails in tabasco'),
 (12749.232323232325, 'poet of apostasy'),
 (13499.187165775402, 'creator of kinder'),
 (14791.1015625, 'consulate in irbil'),
 (17133.58371040724, 'newsworthy and entertaining'),
 (17460.90513833992, 'screened on billboard'),
 (22273.65882352941, 'santos over nepotism'),
 (22896.580498866213, 'hotness of bhut'),
 (28048.311111111107, 'republic-turkey and croatia-spain')]

### common terms influence parameters

While using common_terms, you may have different results also on normal bigrams, 
1. there are no more bigram composed of a word + a common terms
2. as the vocabulary size may increase when using common_terms, you may gain some more bigrams (which could be removed by adjusting the threshold)

In [13]:
# some differences on normal terms
ct_normal_grams = set(g[0].decode("utf-8")
                     for g in bigram_ct.export_phrases(corpus)
                     if len(g[0].split()) <= 2)
no_ct_normal_grams = set(g[0].decode("utf-8") for g in bigram.export_phrases(corpus))
not_found = list(no_ct_normal_grams - ct_normal_grams)
print(len(not_found), " 2-grams not found")
new_found = list(ct_normal_grams - no_ct_normal_grams)
print(len(new_found), " more 2-grams found")
not_found[-20:], new_found[-20:]

242  2-grams not found
1404  more 2-grams found


(['swimming championships',
  'film 28',
  'malaria drug',
  'pakistan collaborate',
  'russians continue',
  'spacecraft crashes',
  'and croatia-spain',
  'most proud',
  'for anzac',
  'in napier',
  'for airasia',
  'burst into',
  'seek asylum',
  'm eastern',
  'for up-to-date',
  'indonesia collaborate',
  'in hotness',
  'as burkina',
  'missile from',
  'does toronto'],
 ['3 000',
  '25 000',
  'obama presidential',
  'jm it',
  'april 5',
  'china russia',
  'has taken',
  'arrest karachi',
  'continue their',
  'buffalo fire',
  'january 13',
  'would take',
  'we must',
  'report released',
  'opposition party',
  'banned from',
  'second train',
  'com november',
  'astronaut eugene',
  've seen'])